In [3]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

# Load the dataset
data = pd.read_csv("Updated_Hotel_Reviews.csv")
data.head(10)


[nltk_data] Downloading package wordnet to C:\Users\Freeware
[nltk_data]     Sys\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Freeware
[nltk_data]     Sys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Freeware
[nltk_data]     Sys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,hotel_name,hotel_address,average_score,total_number_of_reviews,positive_review,negative_review,reviewer_score,lat,lng
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,2.9,52.360576,4.915968
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,No real complaints the hotel was great great ...,No Negative,7.5,52.360576,4.915968
2,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...,7.1,52.360576,4.915968
3,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Great location in nice surroundings the bar a...,My room was dirty and I was afraid to walk ba...,3.8,52.360576,4.915968
4,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Amazing location and building Romantic setting,You When I booked with your company on line y...,6.7,52.360576,4.915968
5,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Good restaurant with modern design great chil...,Backyard of the hotel is total mess shouldn t...,6.7,52.360576,4.915968
6,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,The room is spacious and bright The hotel is ...,Cleaner did not change our sheet and duvet ev...,4.6,52.360576,4.915968
7,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Good location Set in a lovely park friendly s...,Apart from the price for the brekfast Everyth...,10.0,52.360576,4.915968
8,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,No Positive,Even though the pictures show very clean room...,6.5,52.360576,4.915968
9,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,The room was big enough and the bed is good T...,The aircondition makes so much noise and its ...,7.9,52.360576,4.915968


In [6]:
# Replacing "United Kingdom" with "UK"
data['hotel_address'] = data['hotel_address'].str.replace("United Kingdom", "UK")

In [8]:
data["countries"] = data['hotel_address'].apply(lambda x: x.split(' ')[-1])
data.head()

,hotel_name,hotel_address,average_score,total_number_of_reviews,positive_review,negative_review,reviewer_score,lat,lng,countries
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,2.9,52.360576,4.915968,Netherlands
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,No real complaints the hotel was great great ...,No Negative,7.5,52.360576,4.915968,Netherlands
2,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...,7.1,52.360576,4.915968,Netherlands
3,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Great location in nice surroundings the bar a...,My room was dirty and I was afraid to walk ba...,3.8,52.360576,4.915968,Netherlands
4,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,1403,Amazing location and building Romantic setting,You When I booked with your company on line y...,6.7,52.360576,4.915968,Netherlands


In [18]:
print(data.countries.unique())

['netherlands' 'uk' 'france' 'spain' 'italy' 'austria']


In [9]:
# Convert relevant columns to lowercase
data['countries'] = data['countries'].str.lower()

In [11]:
data.drop([ 'lat', 'lng'], axis=1, inplace=True)


In [15]:
def recommend_hotel(location, description, min_user_rating=0):
    description = description.lower()
    tokens = word_tokenize(description)
    stop_words = set(stopwords.words('english'))
    lemm = WordNetLemmatizer()
    
    filtered = {word for word in tokens if word not in stop_words}
    filtered_set = {lemm.lemmatize(word) for word in filtered}
    # Filter based on location and user rating
    country_data = data[(data['countries'] == location.lower()) & (data['average_score'] >= min_user_rating)]
    country_data = country_data.set_index(np.arange(country_data.shape[0]))

    cos_similarities = []
    for i in range(country_data.shape[0]):
        # Process both positive and negative reviews
        review_tokens = word_tokenize(country_data['positive_review'][i]) + \
                        word_tokenize(country_data['negative_review'][i])

        review_set = {lemm.lemmatize(word) for word in review_tokens if word not in stop_words}
        similarity = len(filtered_set.intersection(review_set))
        cos_similarities.append(similarity)

    country_data['similarity'] = cos_similarities

    # Sort by similarity and average score
    country_data = country_data.sort_values(by=['similarity', 'average_score'], ascending=[False, False])
    country_data.drop_duplicates(subset='hotel_name', keep='first', inplace=True)
    country_data.reset_index(drop=True, inplace=True)

    return country_data[["hotel_name", "average_score", "hotel_address"]].head()



In [19]:
recommend_hotel('netherlands', 'I am going for a business trip', min_user_rating=8.0)

,hotel_name,average_score,hotel_address
0,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,8.2,Ndsm Plein 28 Amsterdam Noord 1033 WB Amsterda...
1,Andaz Amsterdam Prinsengracht A Hyatt Hotel,9.3,Prinsengracht 587 Amsterdam City Center 1067 H...
2,Hotel V Fizeaustraat,9.1,Fizeaustraat 2 Oost 1097 SC Amsterdam Netherlands
3,Hotel V Nesplein,9.1,Nes 49 Amsterdam City Center 1012 KD Amsterdam...
4,Hotel Esther a,9.1,Singel 303 309 Amsterdam City Center 1012 WJ A...
